In [2]:
# pip install numpy scipy

import numpy as np
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
from func1 import fun1

# 非线性约束：g(x) = -(x1-1)^2 + x2  >= 0
def g_nonlinear(x):
    x1, x2 = x
    return -(x1 - 1)**2 + x2

# 线性约束：-2*x1 + 3*x2 <= 6
A = np.array([[-2.0, 3.0]])
lin_con = LinearConstraint(A, lb=-np.inf, ub=6.0)

# 非线性约束封装
nl_con = NonlinearConstraint(g_nonlinear, lb=0.0, ub=np.inf)

# ============ 例题 1：给定初始值求解（与 fmincon 默认一致） ============
x0 = np.array([0.0, 0.0])     # 初始值
res_ip = minimize(
    fun1, x0,
    method='trust-constr',     # 对应 fmincon 的 interior-point 思想（信赖域-约束）
    constraints=[lin_con, nl_con],
    options=dict(verbose=0)
)
x_ip, fval_ip = res_ip.x, -res_ip.fun   # 还原为最大值
print("Interior-point 风格（trust-constr）:")
print("x* =", x_ip, "   最大值 f(x*) =", fval_ip)

# ============ 例题 1：使用 SLSQP ============
res_sqp = minimize(
    fun1, x0,
    method='SLSQP',
    constraints=[lin_con, {'type': 'ineq', 'fun': g_nonlinear}],
    options=dict(disp=False, maxiter=1000)
)
x_sqp, fval_sqp = res_sqp.x, -res_sqp.fun
print("\nSLSQP:")
print("x* =", x_sqp, "   最大值 f(x*) =", fval_sqp)

# ============ 改变初始值（与 MATLAB 中 x0=[1,1] 类似） ============
x0_alt = np.array([1.0, 1.0])
res_ip_alt = minimize(
    fun1, x0_alt,
    method='trust-constr',
    constraints=[lin_con, nl_con],
    options=dict(verbose=0)
)
x_ip_alt, fval_ip_alt = res_ip_alt.x, -res_ip_alt.fun
print("\nInterior-point 风格（trust-constr），x0=[1,1]:")
print("x* =", x_ip_alt, "   最大值 f(x*) =", fval_ip_alt)

# =========================================================
# 蒙特卡罗选初值（推荐）
# 在大范围内随机采样可行点，取使 -fun1 最小（即 f 最大）的点作为初值
# 注意：非常大的采样数会较慢，可按需增大
# =========================================================
rng = np.random.default_rng(42)
n = 200000          # 可按需调大（例如 1e6+ 可能很慢）
x1 = rng.uniform(-100, 100, size=n)
x2 = rng.uniform(-100, 100, size=n)

# 约束筛选：-(x1-1)^2 + x2 >= 0  且  -2*x1 + 3*x2 <= 6
mask_feas = (-(x1 - 1)**2 + x2 >= 0) & (-2*x1 + 3*x2 <= 6)

if np.any(mask_feas):
    # 在可行集上评估 f_max
    vals = x1[mask_feas]**2 + x2[mask_feas]**2 - x1[mask_feas]*x2[mask_feas] \
           - 2*x1[mask_feas] - 5*x2[mask_feas]
    idx = np.argmax(vals)
    x0_mc = np.array([x1[mask_feas][idx], x2[mask_feas][idx]])
else:
    # 若没采到可行点（极少见），退回一个合理初值
    x0_mc = np.array([1.0, 1.0])

print("\n蒙特卡罗选取的初始值：", x0_mc)

# 用 MC 初值再求一次
res_mc = minimize(
    fun1, x0_mc,
    method='trust-constr',
    constraints=[lin_con, nl_con],
    options=dict(verbose=0)
)
x_mc, fval_mc = res_mc.x, -res_mc.fun
print("使用 MC 初值的最优解：")
print("x* =", x_mc, "   最大值 f(x*) =", fval_mc)


Interior-point 风格（trust-constr）:
x* = [9.99999992e-01 1.70666678e-09]    最大值 f(x*) = -1.0000000102400006

SLSQP:
x* = [1.00023831e+00 3.79788778e-08]    最大值 f(x*) = -1.0000001710927542

Interior-point 风格（trust-constr），x0=[1,1]:
x* = [9.99999992e-01 1.70666675e-09]    最大值 f(x*) = -1.0000000102400004

蒙特卡罗选取的初始值： [1.46428177 0.32992298]
使用 MC 初值的最优解：
x* = [9.99999992e-01 1.70666677e-09]    最大值 f(x*) = -1.0000000102400006


C:\Users\Administrator\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\Administrator\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
